In [1]:
proj_list = [
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'lcms',
    'libpng', 'libssh',
    'libxml2', 'pcre2',
    'proj4', 're2',
    'sqlite3', 'vorbis',
    'woff2', 'wpantund'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
prefix_np, postfix_np, label_np, label_len_np = data.getTrainData(proj_list, proj_list[target_project])

Getting data for "boringssl" from "c-ares"
Getting data for "boringssl" from "freetype2"
Getting data for "boringssl" from "guetzli"
Getting data for "boringssl" from "harfbuzz"
Getting data for "boringssl" from "lcms"
Getting data for "boringssl" from "libpng"
Getting data for "boringssl" from "libssh"
Getting data for "boringssl" from "libxml2"
Getting data for "boringssl" from "pcre2"
Getting data for "boringssl" from "proj4"
Getting data for "boringssl" from "re2"
Getting data for "boringssl" from "sqlite3"
Getting data for "boringssl" from "vorbis"
Getting data for "boringssl" from "woff2"
Getting data for "boringssl" from "wpantund"


In [6]:
test_prefix, test_postfix, test_label, test_label_len = data.getTestData(proj_list[target_project])

In [7]:
train_prefix, val_prefix, train_postfix, val_postfix, train_label, val_label = train_test_split(
    prefix_np, postfix_np, label_np, test_size = 0.2, random_state = 43
)

In [8]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, train_postfix,
        val_prefix, val_postfix,
        test_prefix, test_postfix,
        train_label, val_label, test_label,
        batch_size=1000
    )

In [9]:
overall_title = 'developing_1'

In [10]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+overall_title+'/tests')

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [12]:
# ====================
# set parameters here
# ====================

title = proj_list[target_project] + '_' + overall_title + '_2'
epochs = 40

max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 0.0

embed_dim = 128
hidden_size = 200
n_layers = 2
output_size = max(token_choices) + 2
dropout = 0.0
max_length = max_len
input_size = max(token_choices) + 2
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "CEL"

teacher_forcing_ratio = 100.0

In [13]:
trainer.set_seed(42)

loss_fn, prefix_pack, postfix_pack, attn_pack = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device
)

In [14]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack,
    teacher_forcing_ratio=teacher_forcing_ratio
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   6.688978   | 95.140738  | 10.755393 | 90.88  | 326.85
   2    |   5.798319   | 97.279747  | 10.098296 | 92.50  | 301.97
   3    |   5.657988   | 97.726408  | 9.761899 | 93.15  | 302.02
   4    |   5.573901   | 98.006934  | 9.588399 | 93.63  | 299.69
   5    |   5.508042   | 98.236205  | 9.389855 | 94.18  | 300.28
   6    |   5.448625   | 98.441433  | 9.365371 | 94.52  | 299.65
   7    |   5.393512   | 98.632415  | 9.239015 | 94.86  | 299.46
   8    |   5.341076   | 98.817621  | 9.025175 | 95.23  | 302.88
   9    |   5.292713   | 98.991494  | 8.937065 | 95.49  | 299.70
  10    |   5.248620   | 99.143858  | 8.881191 | 95.69  | 299.79
  11    |   5.211116   | 99.272544  | 8.813552 | 95.93  | 299.78
  12    |   5.178265   | 99.390751  | 8.730495 | 96.18  | 312.55
  13    |   5.151419   | 99.481989  | 8.749053 | 96

In [15]:
mu.saveModel(overall_title, title, prefix_pack, postfix_pack, attn_pack)

In [16]:
# model = mu.getModel(title)
# print(model)

In [17]:
loss, acc = tester.test(
    test_dataloader=test_dataloader,
    device=device,
    loss_fn=loss_fn,
    prefix_pack=prefix_pack,
    postfix_pack=postfix_pack,
    attn_pack=attn_pack
)

test loss:  21.941512239297708
test acc:  90.56027591085957


In [18]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [19]:
# mu.graphModel(train_dataloader, model, writer)